In [1]:
!pip install -U spacy

     ---------------------------------------- 11.3/11.3 MB 9.9 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 9.6 MB/s eta 0:00:00
     -------------------------------------- 132.6/132.6 KB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 42.8/42.8 KB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 6.6/6.6 MB 9.1 MB/s eta 0:00:00
     -------------------------------------- 112.4/112.4 KB 6.4 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 8.1 MB/s eta 0:00:00
     -------------------------------------- 181.6/181.6 KB 5.5 MB/s eta 0:00:00
     -------------------------------------- 451.8/451.8 KB 9.4 MB/s eta 0:00:00
     ---------------------------------------- 58.6/58.6 KB 3.0 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\K\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
!-m spacy download en_core_web_sm

'-m' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
 !pip install networkx

You should consider upgrading via the 'C:\Users\K\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.



     ---------------------------------------- 2.0/2.0 MB 6.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
candidate_sentences = pd.read_csv("wiki_sentences_v2.csv")

candidate_sentences['sentence'].sample[5]